# Guide Comparison: CRISPR Pipeline vs HonLab FBA

This notebook compares guide quantification and assignment between two pipelines:
1. **UMI Count Comparison** - Raw guide UMI counts per cell
2. **Assignment Comparison** - Binary guide assignments (0/1)

Analyses include:
- Barcode overlap between methods
- Per-guide correlation/agreement
- Per-cell correlation/agreement
- Identification of discordant guides

## Setup

In [1]:
import os
import numpy as np
import pandas as pd
import mudata
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import pearsonr
from matplotlib_venn import venn2

In [42]:
# -----------------------
# Input parameters
# -----------------------

# Path to CRISPR pipeline mudata (.h5mu)
crispr_mudata_path = "/Users/adamklie/Desktop/projects/tf_perturb_seq/datasets/Hon_WTC11-benchmark_TF-Perturb-seq/runs/elated_almeida/inference_mudata.h5mu"

# Path to HonLab FBA output pickle
# For UMI counts: use sgRNA_DF_sgDropNaN.pkl
# For assignments: use final_combined_sgRNA_multiplets_HTO_singlets.pkl
honlab_umi_pickle_path = "/Users/adamklie/Desktop/projects/tf_perturb_seq/datasets/Hon_WTC11-benchmark_TF-Perturb-seq/runs/honlab_internal/LW591_IGVFDS6244NAXC/sgRNA_DF_sgDropNaN.pkl"
honlab_assignment_pickle_path = "/Users/adamklie/Desktop/projects/tf_perturb_seq/datasets/Hon_WTC11-benchmark_TF-Perturb-seq/runs/honlab_internal/LW591_IGVFDS6244NAXC/final_combined_sgRNA_multiplets_HTO_singlets.pkl"

# Batch/lane to filter (e.g., "IGVFDS6244NAXC")
batch_id = "IGVFDS6244NAXC"

# Output directory for figures and tables
output_dir = "/Users/adamklie/Desktop/projects/tf_perturb_seq/datasets/Hon_WTC11-benchmark_TF-Perturb-seq/runs/elated_almeida/pipeline_comparisons"

# Which analyses to run
run_umi_comparison = True
run_assignment_comparison = True

# Number of random cells for per-cell correlation sampling
n_sample_cells = 10000

In [43]:
# Create output directories
fig_dir = os.path.join(output_dir, "figures")
tab_dir = os.path.join(output_dir, "tables")
os.makedirs(fig_dir, exist_ok=True)
os.makedirs(tab_dir, exist_ok=True)

# Plot settings
plt.rcParams.update({
    "pdf.fonttype": 42,
    "ps.fonttype": 42,
    "font.size": 12,
})
sns.set_style("white")

## Helper Functions

In [44]:
def savefig(fig, name: str, close: bool = True):
    """Save figure to output directory."""
    fig.tight_layout()
    fig.savefig(os.path.join(fig_dir, name), dpi=300, bbox_inches="tight")
    if close:
        plt.close(fig)

def ecdf(values: np.ndarray):
    """Compute empirical CDF."""
    x = np.sort(values)
    y = np.arange(1, len(x) + 1) / len(x)
    return x, y

def revcomp(seq: str) -> str:
    """Reverse complement a DNA sequence."""
    complement = str.maketrans('ATCG', 'TAGC')
    return seq.translate(complement)[::-1]

def map_honlab_to_crispr_guide_ids(honlab_cols, crispr_spacer_map, positive_control_map):
    """
    Map HonLab column names to CRISPR pipeline guide IDs.
    
    Handles:
    - Positive controls via manual mapping
    - Non-targeting guides (pass through)
    - Genomic coordinate guides (extract coordinates)
    - Spacer-based mapping via reverse complement
    """
    mapped = []
    for col in honlab_cols:
        # Try positive control map first
        if col in positive_control_map:
            mapped.append(positive_control_map[col])
            continue
        
        # Non-targeting
        if col.startswith("non-targeting"):
            mapped.append(col)
            continue
        
        # Genomic coordinates (already in correct format)
        if col.startswith("chr"):
            # Find matching CRISPR guide by coordinates
            matches = [k for k, v in crispr_spacer_map.items() if col in v]
            if matches:
                mapped.append(crispr_spacer_map[matches[0]])
            else:
                # Try to find by coordinate string in guide_id
                for spacer, guide_id in crispr_spacer_map.items():
                    if col in guide_id:
                        mapped.append(guide_id)
                        break
                else:
                    mapped.append(np.nan)
            continue
        
        # Try spacer-based mapping (extract spacer from column name)
        if "_" in col:
            spacer = col.split("_")[-1]
            rc_spacer = revcomp(spacer)
            if rc_spacer in crispr_spacer_map:
                mapped.append(crispr_spacer_map[rc_spacer])
                continue
            if spacer in crispr_spacer_map:
                mapped.append(crispr_spacer_map[spacer])
                continue
        
        mapped.append(np.nan)
    
    return pd.Index(mapped)

## Load Data

In [45]:
# Load CRISPR pipeline mudata
mdata = mudata.read_h5mu(crispr_mudata_path)
guide = mdata["guide"]

# Filter to batch and extract guide data
guide_batch = guide[guide.obs["batch"] == batch_id].copy()

# Get UMI counts (raw X matrix)
crispr_umi_df = guide_batch.to_df()
crispr_umi_df.index = crispr_umi_df.index.str.split("_").str[0]  # Strip batch suffix

# Get assignments if available
if "guide_assignment" in guide_batch.layers:
    crispr_assign_df = guide_batch.to_df(layer="guide_assignment")
    crispr_assign_df.index = crispr_assign_df.index.str.split("_").str[0]
else:
    crispr_assign_df = (crispr_umi_df > 0).astype(float)

# Get guide metadata and spacer mapping
guide_metadata = guide.var.copy()
crispr_spacer_map = guide.var.set_index("spacer")["guide_id"].to_dict()

print(f"CRISPR data: {crispr_umi_df.shape[0]} cells, {crispr_umi_df.shape[1]} guides")

/Users/adamklie/Desktop/projects/tf_perturb_seq/.venv/lib/python3.10/site-packages/mudata/_core/mudata.py:1598: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("var", axis=0, join_common=join_common)
/Users/adamklie/Desktop/projects/tf_perturb_seq/.venv/lib/python3.10/site-packages/mudata/_core/mudata.py:1461: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("obs", axis=1, join_common=join_common)


CRISPR data: 23275 cells, 416 guides


In [46]:
# Positive control mapping (HonLab name -> CRISPR guide_id)
positive_control_map = {
    "CD81:1": "CD81#strong",
    "CD81:2": "CD81#weak",
    "CD151:1": "CD151#strong",
    "CD151:2": "CD151#weak",
    "CD55": "CD55#strong",
    "TFRC": "TFRC#A",
    "NGFRAP1:1": "NGFRAP1#B",
    "NGFRAP1:2": "NGFRAP1#A",
}

# Reverse mapping for going from CRISPR -> HonLab format
reverse_control_map = {v: k for k, v in positive_control_map.items()}

In [47]:
# Load HonLab UMI counts
if run_umi_comparison and os.path.exists(honlab_umi_pickle_path):
    honlab_umi_raw = pd.read_pickle(honlab_umi_pickle_path).T
    honlab_umi_raw.index = honlab_umi_raw.index.str.split("-").str[0]
    
    # Map column names via reverse complement of spacers
    honlab_umi_raw.columns = pd.Index(
        [revcomp(s) for s in honlab_umi_raw.columns.str.split("_").str[-1]]
    ).map(crispr_spacer_map)
    
    print(f"HonLab UMI data: {honlab_umi_raw.shape[0]} cells, {honlab_umi_raw.shape[1]} guides")
    print(f"Unmapped guides: {honlab_umi_raw.columns.isna().sum()}")
else:
    honlab_umi_raw = None
    print("Skipping HonLab UMI loading")

HonLab UMI data: 44945 cells, 415 guides
Unmapped guides: 0


In [48]:
# Load HonLab assignments
if run_assignment_comparison and os.path.exists(honlab_assignment_pickle_path):
    honlab_assign_raw = pd.read_pickle(honlab_assignment_pickle_path)
    honlab_assign_raw.index = honlab_assign_raw.index.str.split("-").str[0]
    
    # Binarize
    honlab_assign_raw = (honlab_assign_raw > 0).astype(float)
    
    # Map columns: HonLab format -> CRISPR guide_id
    def map_col(col):
        if col in positive_control_map:
            return positive_control_map[col]
        if col.startswith("non-targeting"):
            return col
        if col.startswith("chr"):
            # Find matching guide by coordinate
            for guide_id in guide_metadata.index:
                if col in guide_id:
                    return guide_id
        return np.nan
    
    honlab_assign_raw.columns = honlab_assign_raw.columns.map(map_col)
    
    print(f"HonLab assignment data: {honlab_assign_raw.shape[0]} cells, {honlab_assign_raw.shape[1]} guides")
    print(f"Unmapped guides: {honlab_assign_raw.columns.isna().sum()}")
else:
    honlab_assign_raw = None
    print("Skipping HonLab assignment loading")

HonLab assignment data: 23672 cells, 415 guides
Unmapped guides: 0


## Align Barcodes and Guides

In [49]:
def align_dataframes(crispr_df, honlab_df, name=""):
    """
    Align CRISPR and HonLab dataframes on shared barcodes and guides.
    Returns aligned dataframes and overlap statistics.
    """
    # Remove unmapped columns
    honlab_df = honlab_df.loc[:, honlab_df.columns.notna()].copy()
    
    # Find overlaps
    shared_barcodes = crispr_df.index.intersection(honlab_df.index)
    shared_guides = crispr_df.columns.intersection(honlab_df.columns)
    
    crispr_only_bc = set(crispr_df.index) - set(honlab_df.index)
    honlab_only_bc = set(honlab_df.index) - set(crispr_df.index)
    
    print(f"\n{name} Alignment:")
    print(f"  CRISPR-only barcodes: {len(crispr_only_bc)}")
    print(f"  HonLab-only barcodes: {len(honlab_only_bc)}")
    print(f"  Shared barcodes: {len(shared_barcodes)}")
    print(f"  Shared guides: {len(shared_guides)}")
    
    # Align
    crispr_aligned = crispr_df.loc[shared_barcodes, shared_guides].copy()
    honlab_aligned = honlab_df.loc[shared_barcodes, shared_guides].copy()
    
    # Sort for consistency
    crispr_aligned = crispr_aligned.sort_index().sort_index(axis=1)
    honlab_aligned = honlab_aligned.reindex(index=crispr_aligned.index, columns=crispr_aligned.columns)
    
    overlap_stats = {
        "crispr_only_barcodes": len(crispr_only_bc),
        "honlab_only_barcodes": len(honlab_only_bc),
        "shared_barcodes": len(shared_barcodes),
        "shared_guides": len(shared_guides),
    }
    
    return crispr_aligned, honlab_aligned, overlap_stats

In [50]:
# Align UMI data
if honlab_umi_raw is not None:
    crispr_umi, honlab_umi, umi_overlap = align_dataframes(
        crispr_umi_df, honlab_umi_raw, name="UMI Counts"
    )

# Align assignment data
if honlab_assign_raw is not None:
    crispr_assign, honlab_assign, assign_overlap = align_dataframes(
        crispr_assign_df, honlab_assign_raw, name="Assignments"
    )


UMI Counts Alignment:
  CRISPR-only barcodes: 3
  HonLab-only barcodes: 21673
  Shared barcodes: 23272
  Shared guides: 415

Assignments Alignment:
  CRISPR-only barcodes: 4870
  HonLab-only barcodes: 5267
  Shared barcodes: 18405
  Shared guides: 415


In [51]:
# Barcode overlap Venn diagram
if honlab_umi_raw is not None:
    fig, ax = plt.subplots(figsize=(6, 6))
    venn2(
        subsets=(umi_overlap["crispr_only_barcodes"], 
                 umi_overlap["honlab_only_barcodes"], 
                 umi_overlap["shared_barcodes"]),
        set_labels=("CRISPR Pipeline", "HonLab FBA")
    )
    ax.set_title(f"Barcode Overlap ({batch_id})")
    savefig(fig, "00_barcode_venn_umi.pdf")
    plt.show()

# Barcode overlap Venn diagram for assignments
if honlab_assign_raw is not None:
    fig, ax = plt.subplots(figsize=(6, 6))
    venn2(
        subsets=(assign_overlap["crispr_only_barcodes"], 
                 assign_overlap["honlab_only_barcodes"], 
                 assign_overlap["shared_barcodes"]),
        set_labels=("CRISPR Pipeline", "HonLab FBA")
    )
    ax.set_title(f"Barcode Overlap ({batch_id})")
    savefig(fig, "00_barcode_venn_assignments.pdf")
    plt.show()

---
# Part 1: UMI Count Comparison

Compare raw guide UMI counts between pipelines.

In [52]:
if not run_umi_comparison or honlab_umi_raw is None:
    print("Skipping UMI comparison")

## 1.1 Per-Guide UMI Correlation

In [53]:
if run_umi_comparison and honlab_umi_raw is not None:
    # Per-guide total UMI counts
    crispr_guide_totals = crispr_umi.sum(axis=0).values
    honlab_guide_totals = honlab_umi.sum(axis=0).values
    
    corr = np.corrcoef(crispr_guide_totals, honlab_guide_totals)[0, 1]
    print(f"Per-guide total UMI correlation: {corr:.4f}")
    
    # Scatter plot
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.scatter(crispr_guide_totals, honlab_guide_totals, s=10, alpha=0.5)
    
    # Identity line
    lims = [0, max(crispr_guide_totals.max(), honlab_guide_totals.max())]
    ax.plot(lims, lims, 'r--', lw=1)
    
    ax.set_xlabel("CRISPR Pipeline (total UMI)")
    ax.set_ylabel("HonLab FBA (total UMI)")
    ax.set_title(f"Per-Guide Total UMI (r={corr:.3f})")
    savefig(fig, "01_umi_guide_total_scatter.pdf")
    plt.show()

Per-guide total UMI correlation: 0.9994


In [54]:
if run_umi_comparison and honlab_umi_raw is not None:
    # Per-guide mean UMI counts
    crispr_guide_means = crispr_umi.mean(axis=0).values
    honlab_guide_means = honlab_umi.mean(axis=0).values
    
    corr = np.corrcoef(crispr_guide_means, honlab_guide_means)[0, 1]
    print(f"Per-guide mean UMI correlation: {corr:.4f}")
    
    # Scatter plot
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.scatter(crispr_guide_means, honlab_guide_means, s=10, alpha=0.5)
    
    lims = [0, max(crispr_guide_means.max(), honlab_guide_means.max())]
    ax.plot(lims, lims, 'r--', lw=1)
    
    ax.set_xlabel("CRISPR Pipeline (mean UMI)")
    ax.set_ylabel("HonLab FBA (mean UMI)")
    ax.set_title(f"Per-Guide Mean UMI (r={corr:.3f})")
    savefig(fig, "01_umi_guide_mean_scatter.pdf")
    plt.show()

Per-guide mean UMI correlation: 0.9994


## 1.2 Per-Cell UMI Correlation

In [55]:
if run_umi_comparison and honlab_umi_raw is not None:
    # Per-cell total UMI counts
    crispr_cell_totals = crispr_umi.sum(axis=1).values
    honlab_cell_totals = honlab_umi.sum(axis=1).values
    
    corr = np.corrcoef(crispr_cell_totals, honlab_cell_totals)[0, 1]
    print(f"Per-cell total UMI correlation: {corr:.4f}")
    
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.scatter(crispr_cell_totals, honlab_cell_totals, s=4, alpha=0.3)
    
    lims = [0, max(crispr_cell_totals.max(), honlab_cell_totals.max())]
    ax.plot(lims, lims, 'r--', lw=1)
    
    ax.set_xlabel("CRISPR Pipeline (total UMI/cell)")
    ax.set_ylabel("HonLab FBA (total UMI/cell)")
    ax.set_title(f"Per-Cell Total UMI (r={corr:.4f})")
    savefig(fig, "02_umi_cell_total_scatter.pdf")
    plt.show()

Per-cell total UMI correlation: 0.9921


## 1.3 Per-Cell Per-Guide Correlation Distribution

In [56]:
if run_umi_comparison and honlab_umi_raw is not None:
    # Sample cells and compute per-cell guide correlations
    n_cells = min(n_sample_cells, len(crispr_umi))
    cell_indices = np.random.choice(len(crispr_umi), size=n_cells, replace=False)
    
    cell_corrs = []
    for idx in cell_indices:
        crispr_vals = crispr_umi.iloc[idx].values
        honlab_vals = honlab_umi.iloc[idx].values
        if np.std(crispr_vals) > 0 and np.std(honlab_vals) > 0:
            cell_corrs.append(pearsonr(crispr_vals, honlab_vals)[0])
    
    print(f"Median per-cell correlation: {np.median(cell_corrs):.4f}")
    print(f"Mean per-cell correlation: {np.mean(cell_corrs):.4f}")
    
    fig, ax = plt.subplots(figsize=(8, 4))
    ax.hist(cell_corrs, bins=50, edgecolor='black', alpha=0.7)
    ax.axvline(np.median(cell_corrs), color='red', linestyle='--', label=f'Median={np.median(cell_corrs):.3f}')
    ax.set_xlabel("Pearson correlation (across guides)")
    ax.set_ylabel("# cells")
    ax.set_title(f"Per-Cell UMI Correlation Distribution (n={n_cells})")
    ax.legend()
    savefig(fig, "03_umi_cell_guide_corr_hist.pdf")
    plt.show()

Median per-cell correlation: 1.0000
Mean per-cell correlation: 0.9857


In [57]:
if run_umi_comparison and honlab_umi_raw is not None:
    # Per-guide correlation (across cells)
    guide_corrs = []
    for guide in crispr_umi.columns:
        crispr_vals = crispr_umi[guide].values
        honlab_vals = honlab_umi[guide].values
        if np.std(crispr_vals) > 0 and np.std(honlab_vals) > 0:
            guide_corrs.append(pearsonr(crispr_vals, honlab_vals)[0])
        else:
            guide_corrs.append(np.nan)
    
    guide_corr_df = pd.DataFrame({
        "guide_id": crispr_umi.columns,
        "correlation": guide_corrs
    }).dropna()
    
    print(f"Median per-guide correlation: {guide_corr_df['correlation'].median():.4f}")
    
    fig, ax = plt.subplots(figsize=(8, 4))
    ax.hist(guide_corr_df['correlation'].values, bins=50, edgecolor='black', alpha=0.7)
    ax.axvline(guide_corr_df['correlation'].median(), color='red', linestyle='--', 
               label=f'Median={guide_corr_df["correlation"].median():.3f}')
    ax.set_xlabel("Pearson correlation (across cells)")
    ax.set_ylabel("# guides")
    ax.set_title("Per-Guide UMI Correlation Distribution")
    ax.legend()
    savefig(fig, "03_umi_guide_cell_corr_hist.pdf")
    plt.show()
    
    # Save low-correlation guides
    low_corr = guide_corr_df[guide_corr_df['correlation'] < 0.9].sort_values('correlation')
    low_corr.to_csv(os.path.join(tab_dir, "umi_low_correlation_guides.csv"), index=False)
    print(f"\nGuides with correlation < 0.9: {len(low_corr)}")
    if len(low_corr) > 0:
        print(low_corr.head(10))

Median per-guide correlation: 0.9982

Guides with correlation < 0.9: 7
                                guide_id  correlation
297    SMARCC1#chr3:47781830-47781848(-)     0.274684
379  TFCP2L1#chr2:121284793-121284811(-)     0.358031
149       NANOG#chr12:7789786-7789804(-)     0.364326
227     POU5F1#chr6:31170702-31170720(-)     0.750941
351     TCF12#chr15:56918702-56918720(-)     0.772793
62      CTNNB1#chr3:41199540-41199558(+)     0.832403
330     SOX2#chr3:181711908-181711926(+)     0.899493

Guides with correlation < 0.9: 7
                                guide_id  correlation
297    SMARCC1#chr3:47781830-47781848(-)     0.274684
379  TFCP2L1#chr2:121284793-121284811(-)     0.358031
149       NANOG#chr12:7789786-7789804(-)     0.364326
227     POU5F1#chr6:31170702-31170720(-)     0.750941
351     TCF12#chr15:56918702-56918720(-)     0.772793
62      CTNNB1#chr3:41199540-41199558(+)     0.832403
330     SOX2#chr3:181711908-181711926(+)     0.899493


---
# Part 2: Assignment Comparison

Compare binary guide assignments between pipelines.

In [58]:
if not run_assignment_comparison or honlab_assign_raw is None:
    print("Skipping assignment comparison")

## 2.1 Cell-Level Assignment Summary

In [59]:
if run_assignment_comparison and honlab_assign_raw is not None:
    # Guides per cell
    n_guides_crispr = crispr_assign.sum(axis=1).astype(int)
    n_guides_honlab = honlab_assign.sum(axis=1).astype(int)
    
    cell_summary = pd.DataFrame({
        "n_guides_crispr": n_guides_crispr,
        "n_guides_honlab": n_guides_honlab,
        "any_crispr": (n_guides_crispr > 0),
        "any_honlab": (n_guides_honlab > 0),
        "exactly1_crispr": (n_guides_crispr == 1),
        "exactly1_honlab": (n_guides_honlab == 1),
    })
    cell_summary.to_csv(os.path.join(tab_dir, "cell_assignment_summary.csv"))
    
    # Category counts
    def cat_counts(n):
        return pd.Series({
            "0 guides": int((n == 0).sum()),
            "1 guide": int((n == 1).sum()),
            ">1 guides": int((n > 1).sum()),
        })
    
    cats = pd.DataFrame({
        "CRISPR": cat_counts(n_guides_crispr),
        "HonLab": cat_counts(n_guides_honlab),
    })
    cats_frac = cats / cats.sum(axis=0)
    
    print("Guides per cell distribution:")
    print(cats_frac.round(3))

Guides per cell distribution:
           CRISPR  HonLab
0 guides    0.001   0.000
1 guide     0.586   0.632
>1 guides   0.413   0.368


In [60]:
if run_assignment_comparison and honlab_assign_raw is not None:
    # Bar plot of categories
    fig, ax = plt.subplots(figsize=(6, 4))
    x = np.arange(len(cats_frac))
    w = 0.35
    ax.bar(x - w/2, cats_frac["CRISPR"].values, width=w, label="CRISPR", color="#1f77b4")
    ax.bar(x + w/2, cats_frac["HonLab"].values, width=w, label="HonLab", color="#ff7f0e")
    ax.set_xticks(x)
    ax.set_xticklabels(cats_frac.index)
    ax.set_ylabel("Fraction of cells")
    ax.set_title("Guides per Cell Category")
    ax.legend(frameon=False)
    savefig(fig, "04_assign_guides_per_cell_bar.pdf")
    plt.show()

In [61]:
if run_assignment_comparison and honlab_assign_raw is not None:
    # Histogram of guides per cell
    fig, ax = plt.subplots(figsize=(8, 4))
    bins = np.arange(0, max(n_guides_crispr.max(), n_guides_honlab.max()) + 2) - 0.5
    ax.hist(n_guides_crispr.values, bins=bins, alpha=0.6, label="CRISPR", color="#1f77b4")
    ax.hist(n_guides_honlab.values, bins=bins, alpha=0.6, label="HonLab", color="#ff7f0e")
    ax.set_xlabel("# guides per cell")
    ax.set_ylabel("# cells")
    ax.set_title("Distribution of Guides per Cell")
    ax.legend(frameon=False)
    savefig(fig, "04_assign_guides_per_cell_hist.pdf")
    plt.show()

In [62]:
if run_assignment_comparison and honlab_assign_raw is not None:
    # Confusion matrix: any guide assigned
    conf = pd.crosstab(
        cell_summary["any_crispr"].map({False: "No guide", True: ">=1 guide"}),
        cell_summary["any_honlab"].map({False: "No guide", True: ">=1 guide"}),
        rownames=["CRISPR"],
        colnames=["HonLab"],
    )
    conf = conf.reindex(index=["No guide", ">=1 guide"], columns=["No guide", ">=1 guide"], fill_value=0)
    conf.to_csv(os.path.join(tab_dir, "cell_any_guide_confusion.csv"))
    
    fig, ax = plt.subplots(figsize=(5, 4))
    im = ax.imshow(conf.values, cmap="Blues")
    ax.set_xticks(range(len(conf.columns)))
    ax.set_yticks(range(len(conf.index)))
    ax.set_xticklabels(conf.columns)
    ax.set_yticklabels(conf.index)
    ax.set_xlabel("HonLab")
    ax.set_ylabel("CRISPR")
    ax.set_title("Cell-Level Agreement: Any Guide")
    
    for i in range(conf.shape[0]):
        for j in range(conf.shape[1]):
            ax.text(j, i, f"{conf.iloc[i, j]:,}", ha="center", va="center", fontsize=12)
    
    fig.colorbar(im, ax=ax, shrink=0.8, label="# cells")
    savefig(fig, "05_assign_any_guide_confusion.pdf")
    plt.show()

## 2.2 Per-Guide Jaccard Overlap

In [63]:
if run_assignment_comparison and honlab_assign_raw is not None:
    # Compute per-guide Jaccard
    crispr_bin = (crispr_assign > 0).astype(int)
    honlab_bin = (honlab_assign > 0).astype(int)
    
    intersection = (crispr_bin & honlab_bin).sum(axis=0)
    union = (crispr_bin | honlab_bin).sum(axis=0)
    jaccard = (intersection / union.replace(0, np.nan)).fillna(0)
    
    guide_summary = pd.DataFrame({
        "n_cells_crispr": crispr_bin.sum(axis=0),
        "n_cells_honlab": honlab_bin.sum(axis=0),
        "intersection": intersection,
        "union": union,
        "jaccard": jaccard,
        "delta": crispr_bin.sum(axis=0) - honlab_bin.sum(axis=0),
    })
    guide_summary["abs_delta"] = guide_summary["delta"].abs()
    
    # Merge with guide metadata if available
    if guide_metadata is not None:
        guide_summary = guide_summary.merge(
            guide_metadata[["targeting", "type", "intended_target_name"]],
            left_index=True, right_index=True, how="left"
        )
    
    guide_summary.to_csv(os.path.join(tab_dir, "guide_assignment_summary.csv"))
    
    print(f"Median Jaccard: {guide_summary['jaccard'].median():.4f}")
    print(f"Mean Jaccard: {guide_summary['jaccard'].mean():.4f}")

Median Jaccard: 0.9451
Mean Jaccard: 0.9289


In [64]:
if run_assignment_comparison and honlab_assign_raw is not None:
    # Jaccard histogram
    fig, ax = plt.subplots(figsize=(8, 4))
    ax.hist(guide_summary["jaccard"].values, bins=40, edgecolor="black", alpha=0.7)
    ax.axvline(guide_summary["jaccard"].median(), color="red", linestyle="--",
               label=f"Median={guide_summary['jaccard'].median():.3f}")
    ax.set_xlabel("Jaccard overlap")
    ax.set_ylabel("# guides")
    ax.set_title("Per-Guide Assignment Overlap")
    ax.legend()
    savefig(fig, "06_assign_jaccard_hist.pdf")
    plt.show()

In [65]:
if run_assignment_comparison and honlab_assign_raw is not None:
    # Scatter: cells per guide
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.scatter(
        np.log1p(guide_summary["n_cells_crispr"]),
        np.log1p(guide_summary["n_cells_honlab"]),
        s=12, alpha=0.5, c=guide_summary["jaccard"], cmap="viridis"
    )
    lims = [0, max(np.log1p(guide_summary["n_cells_crispr"]).max(),
                  np.log1p(guide_summary["n_cells_honlab"]).max())]
    ax.plot(lims, lims, 'r--', lw=1)
    ax.set_xlabel("log1p(# cells) - CRISPR")
    ax.set_ylabel("log1p(# cells) - HonLab")
    ax.set_title("Per-Guide Assignment Counts")
    plt.colorbar(ax.collections[0], ax=ax, label="Jaccard")
    savefig(fig, "06_assign_guide_counts_scatter.pdf")
    plt.show()

In [66]:
if run_assignment_comparison and honlab_assign_raw is not None:
    # Low Jaccard guides
    low_jaccard = guide_summary[guide_summary["jaccard"] < 0.7].sort_values("jaccard")
    low_jaccard.to_csv(os.path.join(tab_dir, "low_jaccard_guides.csv"))
    
    print(f"\nGuides with Jaccard < 0.7: {len(low_jaccard)}")
    if len(low_jaccard) > 0:
        print(low_jaccard[["n_cells_crispr", "n_cells_honlab", "jaccard"]].head(15))


Guides with Jaccard < 0.7: 11
                                     n_cells_crispr  n_cells_honlab   jaccard
CTNNB1#chr3:41199540-41199558(+)                  0               0  0.000000
POU5F1#chr6:31170169-31170187(-)                  0               0  0.000000
POU5F1#chr6:31170702-31170720(-)                  0               0  0.000000
ARNT#chr1:150876336-150876354(-)                  2               1  0.500000
POU5F1#chr6:31170199-31170217(-)                  2               1  0.500000
SMARCC1#chr3:47781830-47781848(-)                 1               2  0.500000
SMARCD1#chr12:50085598-50085616(+)                2               1  0.500000
MYC#chr8:127736042-127736060(+)                  17              29  0.586207
NANOG#chr12:7789847-7789865(-)                    3               2  0.666667
NANOG#chr12:7789883-7789901(+)                    2               3  0.666667
TCF7L2#chr10:112950266-112950284(-)              10              15  0.666667


## 2.3 Single-Guide Cell Agreement

In [67]:
if run_assignment_comparison and honlab_assign_raw is not None:
    # For cells with exactly 1 guide in both methods, check if they match
    def get_single_guide(df):
        row_sums = df.sum(axis=1)
        guides = df.idxmax(axis=1)
        return guides.where(row_sums == 1, other=np.nan)
    
    crispr_single = get_single_guide(crispr_assign)
    honlab_single = get_single_guide(honlab_assign)
    
    single_df = pd.DataFrame({
        "crispr_guide": crispr_single,
        "honlab_guide": honlab_single,
    })
    
    # Categorize
    mask_c = single_df["crispr_guide"].notna()
    mask_h = single_df["honlab_guide"].notna()
    
    single_df["category"] = "other"
    single_df.loc[mask_c & ~mask_h, "category"] = "single_only_crispr"
    single_df.loc[~mask_c & mask_h, "category"] = "single_only_honlab"
    single_df.loc[(mask_c & mask_h) & (single_df["crispr_guide"] == single_df["honlab_guide"]), "category"] = "single_both_same"
    single_df.loc[(mask_c & mask_h) & (single_df["crispr_guide"] != single_df["honlab_guide"]), "category"] = "single_both_diff"
    
    single_df.to_csv(os.path.join(tab_dir, "single_guide_comparison.csv"))
    
    # Summary
    cat_counts = single_df["category"].value_counts()
    print("Single-guide cell categories:")
    print(cat_counts)

Single-guide cell categories:
category
single_both_same      10765
other                  6760
single_only_honlab      867
single_only_crispr       13
Name: count, dtype: int64


In [68]:
if run_assignment_comparison and honlab_assign_raw is not None:
    # Bar plot of single-guide categories
    order = ["single_both_same", "single_both_diff", "single_only_crispr", "single_only_honlab", "other"]
    cat_counts = cat_counts.reindex(order).fillna(0)
    
    fig, ax = plt.subplots(figsize=(8, 4))
    ax.bar(range(len(cat_counts)), cat_counts.values, color="#2ca02c")
    ax.set_xticks(range(len(cat_counts)))
    ax.set_xticklabels(cat_counts.index, rotation=30, ha="right")
    ax.set_ylabel("# cells")
    ax.set_title("Single-Guide Cell Agreement")
    savefig(fig, "07_assign_single_guide_categories.pdf")
    plt.show()

---
## Summary

In [69]:
# Compile run summary
summary = {
    "batch_id": batch_id,
}

if run_umi_comparison and honlab_umi_raw is not None:
    summary.update({
        "umi_n_cells": int(crispr_umi.shape[0]),
        "umi_n_guides": int(crispr_umi.shape[1]),
        "umi_guide_total_corr": float(np.corrcoef(crispr_umi.sum(0), honlab_umi.sum(0))[0, 1]),
        "umi_cell_total_corr": float(np.corrcoef(crispr_umi.sum(1), honlab_umi.sum(1))[0, 1]),
    })

if run_assignment_comparison and honlab_assign_raw is not None:
    summary.update({
        "assign_n_cells": int(crispr_assign.shape[0]),
        "assign_n_guides": int(crispr_assign.shape[1]),
        "assign_frac_any_crispr": float((n_guides_crispr > 0).mean()),
        "assign_frac_any_honlab": float((n_guides_honlab > 0).mean()),
        "assign_frac_exactly1_crispr": float((n_guides_crispr == 1).mean()),
        "assign_frac_exactly1_honlab": float((n_guides_honlab == 1).mean()),
        "assign_median_jaccard": float(guide_summary["jaccard"].median()),
    })

summary_df = pd.Series(summary)
summary_df.to_csv(os.path.join(tab_dir, "run_summary.csv"), header=False)
print("\nRun Summary:")
print(summary_df.to_string())


Run Summary:
batch_id                       IGVFDS6244NAXC
umi_n_cells                             23272
umi_n_guides                              415
umi_guide_total_corr                 0.999433
umi_cell_total_corr                  0.992115
assign_n_cells                          18405
assign_n_guides                           415
assign_frac_any_crispr               0.998968
assign_frac_any_honlab                    1.0
assign_frac_exactly1_crispr          0.585602
assign_frac_exactly1_honlab          0.632002
assign_median_jaccard                0.945055


In [70]:
print(f"\nOutputs saved to:")
print(f"  Figures: {fig_dir}")
print(f"  Tables: {tab_dir}")


Outputs saved to:
  Figures: /Users/adamklie/Desktop/projects/tf_perturb_seq/datasets/Hon_WTC11-benchmark_TF-Perturb-seq/runs/elated_almeida/pipeline_comparisons/figures
  Tables: /Users/adamklie/Desktop/projects/tf_perturb_seq/datasets/Hon_WTC11-benchmark_TF-Perturb-seq/runs/elated_almeida/pipeline_comparisons/tables


# DONE!

---